In [1]:
# %cd test
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import time
import shutil
import torch.nn as nn
from skimage import io
import torchvision
import cv2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from albumentations.pytorch import ToTensorV2
from torchvision import utils
import albumentations as A
from albumentations import (HorizontalFlip, ShiftScaleRotate, VerticalFlip, Normalize,Flip,
                            Compose, GaussNoise)
from shutil import copyfile
from torchvision import transforms
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = DEVICE
BASE_DIR = "../input/tensorflow-great-barrier-reef/train_images/"

num_epochs = 12
width = 852
height = 480

In [2]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")

# Turn annotations from strings into lists of dictionaries
df['annotations1'] = df['annotations'].apply(eval)

# Create the image path for the row
df['image_path'] = "video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

length = lambda x: len(x) 

df["no_of_bbox"] = df["annotations1"].apply(length)
df['annotations'] = df['annotations'].apply(eval)

df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,annotations1,image_path,no_of_bbox
0,0,40258,0,0,0-0,[],[],video_0/0.jpg,0
1,0,40258,1,1,0-1,[],[],video_0/1.jpg,0
2,0,40258,2,2,0-2,[],[],video_0/2.jpg,0
3,0,40258,3,3,0-3,[],[],video_0/3.jpg,0
4,0,40258,4,4,0-4,[],[],video_0/4.jpg,0


In [3]:
val_df = df[df["video_id"]==2][df["no_of_bbox"]>0]
val_df.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(677, 9)

In [4]:
type(val_df)

pandas.core.frame.DataFrame

In [5]:
# df = pd.read_csv("../input/reef-cv-strategy-subsequences-dataframes/train-validation-split/train-0.1.csv")

# # Turn annotations from strings into lists of dictionaries
# df['annotations'] = df['annotations'].apply(eval)

# # Create the image path for the row
# df['image_path'] = "video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

# df_train, df_val = df[df['is_train']], df[~df['is_train']]

# df_train = df_train[df_train.annotations.str.len() > 0 ].reset_index(drop=True)
# df_val = df_val[df_val.annotations.str.len() > 0 ].reset_index(drop=True)

# df_train.shape[0], df_val.shape[0]

In [6]:
class ReefDataset:
    
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def can_augment(self, boxes):
        """ Check if bounding boxes are OK to augment
        
        
        For example: image_id 1-490 has a bounding box that is partially outside of the image
        It breaks albumentation
        Here we check the margins are within the image to make sure the augmentation can be applied
        """
        
        box_outside_image = ((boxes[:, 0] < 0).any() or (boxes[:, 1] < 0).any() 
                             or (boxes[:, 2] > 1280).any() or (boxes[:, 3] > 720).any())
        return not box_outside_image
        
    def get_boxes(self, row):
        """Returns the bboxes for a given row as a 3D matrix with format [x_min, y_min, x_max, y_max]"""
        
        boxes = pd.DataFrame(row['annotations'], columns=['x', 'y', 'width', 'height']).astype(float).values
        
        # Change from [x_min, y_min, w, h] to [x_min, y_min, x_max, y_max]
        boxes[:, 2] = np.clip(boxes[:, 0] + boxes[:, 2],0,1280)
        boxes[:, 3] = np.clip(boxes[:, 1] + boxes[:, 3],0,720)
        return boxes
    
    def get_image(self, row):
        """Gets the image for a given row"""
        print(f'{BASE_DIR}{row["image_path"]}')
        image = cv2.imread(f'{BASE_DIR}{row["image_path"]}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0
        return image,f'{BASE_DIR}{row["image_path"]}'
    
    def __getitem__(self, i):

        row = self.df.iloc[i]
        image,pth = self.get_image(row)
        boxes = self.get_boxes(row)
        boxes1 = boxes
        n_boxes = boxes.shape[0]
        
        # Calculate the area
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'area': torch.as_tensor(area, dtype=torch.float32),
            
            'image_id': torch.tensor([i]),
            
            # There is only one class
            'labels': torch.ones((n_boxes,), dtype=torch.int64), #unchanged
            
            # Suppose all instances are not crowd
            'iscrowd': torch.zeros((n_boxes,), dtype=torch.int64) #unchanged   
        }
        
#         if self.transforms is not None:
#             image, target = self.transforms(image, target)
#         return image, target

        if self.transforms and self.can_augment(boxes):
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': target['labels']
            }
            sample = self.transforms(**sample)
            image = sample['image']
#             target['boxes'] = torch.as_tensor(sample['bboxes'])
#             if len(tensor.size()) != 0:
#                 target['boxes'] = target['boxes'].int()
#                 target['boxes'] = target['boxes'].to(torch.float32)
#                 print(f"{torch.tensor([i])} : {target['boxes'].shape}")
#                 print(f"{torch.tensor([i])} : {target['boxes']}")
#                 print("#"*80)
#                 return image, target
#             else:
#                 continue
                
#             print(*sample['bboxes'][0])
            if n_boxes > 0:
#                 print(torch.tensor(sample['bboxes']))
                target['boxes'] = torch.as_tensor(sample['bboxes']).int()
                target['boxes'] = target['boxes'].to(torch.float32)
#                 print(f"{torch.tensor([i])} : {target['boxes'].shape}")
#                 print(f"{torch.tensor([i])} : {target['boxes']}")
#                 print("#"*80)

        else:
            image = ToTensorV2(p=1.0)(image=image)['image']
            
#         if target['boxes'].shape != (0):
        return image, target, pth, boxes1

    def __len__(self):
        return len(self.df)

In [7]:
def get_train_transform():
    return A.Compose([
#         A.Resize(height=height, width=width, p=1.0),
#         A.Flip(0.0),
        A.HorizontalFlip(0.8), # Same with transforms.RandomHorizontalFlip()

#         A.VerticalFlip(0.1), 

#         A.Cutout(num_holes=25, max_h_size=6, max_w_size=6, fill_value=0, p=0.5),

        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


In [8]:
ds_train = ReefDataset(val_df,get_train_transform())
# ds_val = ReefDataset(df_val,get_valid_transform())

In [9]:
# idx = 34
# image, targets, pth, bboxes = ds_train[idx]
# boxes = targets['boxes'].cpu().numpy().astype(np.int32)

# img2 = image.permute(1,2,0).cpu().numpy()
# fig, ax = plt.subplots(1, 1, figsize=(16, 18))

# for box in boxes:
#     print(box)
#     cv2.rectangle(img2,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 2)
    
# ax.set_axis_off()

# ax.imshow(img2);

In [10]:
!mkdir -p /kaggle/working/images/
!mkdir -p /kaggle/working/labels/

In [11]:
from PIL import Image 

for i in range(len(val_df)):
    
    image, targets, pths, _ = ds_train[i]
    boxes = targets['boxes'].cpu().numpy().astype(np.int32)

    img1 = image.permute(1,2,0).cpu().numpy().astype(np.uint8)
    from_arr = Image.fromarray(img1)
#     fig, ax = plt.subplots(1, 1, figsize=(16, 18))
    
    r = ''
    for an in boxes:
#         print("an",an)
        r += '{} {} {} {}\n'.format(an[0] , an[1], an[2], an[3])
#         box = an
#         cv2.rectangle(img1,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 2)
#     print(r)
#     ax.set_axis_off()
#     ax.imshow(img1);
#     break
#     print(pths.split("/")[5].split(".")[0])
#     break
    from_arr.save(f'/kaggle/working/images/{pths.split("/")[5].split(".")[0]}.jpg')
#     cv2.imwrite(f'/kaggle/working/images/{pths.split("/")[5].split(".")[0]}.jpg',img1)
    with open(f'/kaggle/working/labels/{pths.split("/")[5].split(".")[0]}.txt', 'w') as fp:
        fp.write(r)
        

../input/tensorflow-great-barrier-reef/train_images/video_2/4718.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4719.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4720.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4721.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4722.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4723.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4724.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4725.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4726.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4727.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4728.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4729.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4730.jpg
../input/tensorflow-great-barrier-reef/train_images/video_2/4731.jpg
../input/tensorflow-great-barrier-

In [12]:
!ls -A ./images

10619.jpg  5405.jpg  5494.jpg  5734.jpg  5819.jpg  5904.jpg  6309.jpg  6407.jpg
10620.jpg  5406.jpg  5495.jpg  5735.jpg  5820.jpg  5905.jpg  6310.jpg  6408.jpg
10621.jpg  5407.jpg  5496.jpg  5736.jpg  5821.jpg  5906.jpg  6311.jpg  6409.jpg
10622.jpg  5408.jpg  5497.jpg  5737.jpg  5822.jpg  5907.jpg  6312.jpg  6410.jpg
10623.jpg  5409.jpg  5498.jpg  5738.jpg  5823.jpg  5908.jpg  6313.jpg  6411.jpg
10624.jpg  5410.jpg  5499.jpg  5739.jpg  5824.jpg  5909.jpg  6314.jpg  6412.jpg
10625.jpg  5411.jpg  5500.jpg  5740.jpg  5825.jpg  5910.jpg  6315.jpg  6413.jpg
10626.jpg  5412.jpg  5501.jpg  5741.jpg  5826.jpg  5911.jpg  6329.jpg  6414.jpg
10627.jpg  5413.jpg  5502.jpg  5742.jpg  5827.jpg  5912.jpg  6330.jpg  6415.jpg
10628.jpg  5414.jpg  5503.jpg  5743.jpg  5828.jpg  5913.jpg  6331.jpg  6416.jpg
10629.jpg  5415.jpg  5504.jpg  5744.jpg  5829.jpg  5914.jpg  6332.jpg  6417.jpg
10630.jpg  5416.jpg  5505.jpg  5745.jpg  5830.jpg  5915.jpg  6333.jpg  6418.jpg
10631.jpg  5417.jpg  5506.jpg  5746.jpg 

In [13]:
# plt.imshow(plt.imread("./geeks.jpg"))

In [14]:
# print(bbox)

In [15]:
# annots = val_df.iloc[idx]["annotations"]
# imgpth = val_df.iloc[idx]["image_path"]

In [16]:
# annots

In [17]:
# img_pth = "../input/tensorflow-great-barrier-reef/train_images/video_2/5432.jpg"
# fig, ax = plt.subplots(1, 1, figsize=(16, 18))
# img_og = plt.imread(pth)
# for bbox in bboxes:
#     print(bbox)
#     cv2.rectangle(img_og,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (220, 0, 0), 2)
    
# ax.set_axis_off()

# ax.imshow(img_og);